In [18]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import FGA
from deeprobust.graph.utils import *
from deeprobust.graph.data import Dataset
from tqdm import tqdm
import argparse
from experiments import split_dataset
from DistributedDefense import TwoPartyCNGCN
import networkx as nx
from scipy.sparse import csr_matrix
import Mahsa_backdoor_V0 as backdoor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
#import random # for random choice of nodes

In [ ]:
################################# V0: adding edges - Attack repeatedly - consider crypto'Graph ###############################
# Define the number of repeats
num_repeats = 1
accuracy_test_attack_1_values = []
accuracy_test_clean_1_values = []
accuracy_test_attack_2_values = []
accuracy_test_clean_2_values = []
accuracy_test_attack_3_values = []
accuracy_test_clean_3_values = []

################################ Data loading #######################################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load data
dataset = "polblogs"
# Use the current directory for windows
data = Dataset(root='.', name=dataset)
#data = Dataset(root='/tmp/', name=dataset): this is for unix-based systems

for _ in range(num_repeats):
    
    #############################  preprocessing ############################
    adj, features, labels = data.adj, data.features, data.labels
    idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

    #split idx_test into two parts randomly
    test_size = 0.10  # 10% for test_attack, 90% for test_clean
    # test_size = 1- 1 / len(idx_test)  # 1 node for test_attack, the rest for test_clean

    idx_test_clean , idx_test_attack = train_test_split(idx_test, test_size=test_size, random_state=42)


    # Split graph into two graphs 
    proportion_of_common_links = 0.5
    adj1, adj2 = split_dataset(adj, proportion_of_common_links) 


# print ("idx_test_clean",len (idx_test_clean))
# print ("idx_test_attack",len (idx_test_attack))
# print ("idx_test",len (idx_test))
# print ("train",len (idx_train))
# print ("val",len (idx_val))


    ############################ tarin model initially and test accuracy ###########################
    # Perform evaluation before attack to find the baseline accuracy

    # Train GCN model 
    # 1. Model definition
    model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device, dropout=0.5)
    model = model.to(device)
    # 2. Train model - patience= number of epochs to wait before early stopping , train_iters= number of iterations
    model.fit(features, adj, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200)
    #model.fit(features, adj, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200,metric=metric,
               #object=object, initialize=True, verbose=False )
    # 3. Evaluate model
    model.eval()
    output = model.test(idx_test)
    #acc_test = accuracy(output, labels, idx_test)

    # accuracies = model.test(idx_test) 
    # print("Test accuracy: ", accuracies)

    # output_1 = model.test(idx_test_attack)
    # output_2 = model.test(idx_test_clean)

    accuracy_test_attack_1 = model.test(idx_test_attack) 
    accuracy_test_clean_1 = model.test(idx_test_clean)

    print("Test accuracy on attack set: ", accuracy_test_attack_1)
    print("Test accuracy on clean set: ", accuracy_test_clean_1)

    
    ################################ Mahsa attack ##############################
    # Perform the attack
    # version 0: attack on the whole graph - on idx_test_attack nodes

    modified_adj1 =  adj1.copy()

    # Create a NetworkX graph from the adjacency matrix
    graph = nx.from_scipy_sparse_array(modified_adj1)

    # Add labels to the graph
    for node_id, label in enumerate(labels):
        graph.nodes[node_id]['label'] = label
    # print(f"lenght of the labels : {len(labels)}") 
    print(f"graph edges : {graph.number_of_edges()}")
    print(f"graph nodes : {graph.number_of_nodes()}")
    # Set the budget for the attack
    budget = 5

    #attacked_graph is initially set to graph, and then updated after each attack
    # This means that each attack is performed on the graph resulting from the previous attacks.
    attacked_graph = graph
    for target_node in idx_test_attack:
        non_neighbor_opposit = backdoor.find_non_neighbor_opposit_label(attacked_graph, target_node)
        max_same_min_opposit_label_neighbors = backdoor.find_max_same_min_opposit_label_neighbors(attacked_graph, non_neighbor_opposit)
        nodes_for_attack = backdoor.nodes_for_attack(attacked_graph, target_node, max_same_min_opposit_label_neighbors, budget)
        attacked_graph = backdoor.insert_edge(attacked_graph, target_node, nodes_for_attack)
        backdoor.evaluate_graph(attacked_graph, nodes_for_attack, target_node, budget)
        
    # Convert the graph to a CSR matrix
    modified_adj1 =backdoor.convert(attacked_graph)
    print(attacked_graph)
    print(f"total inserted edges : {attacked_graph.number_of_edges() - graph.number_of_edges()}")

    ################################ evaluation after attack ##############################
    # accuracy after attack
    model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device, dropout=0.5)
    model = model.to(device)
    #data = data.to(device)
    model.fit(features, modified_adj1, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200)
    model.eval()

    # output = model.test(idx_test)

    #acc_test = accuracy(output, labels, idx_test)

    # output_1 = model.test(idx_test_attack)
    # output_2 = model.test(idx_test_clean)

    accuracy_test_attack_2 = model.test(idx_test_attack) 
    accuracy_test_clean_2 = model.test(idx_test_clean)

    print("Test accuracy on attack set: ", accuracy_test_attack_2)
    print("Test accuracy on clean set: ", accuracy_test_clean_2)

    ############################ Crypto'Graph defense ###########################
    # Perform Crypto'Graph distributed defense

    threshold = 2               # threshold for dropping dissimilar edges
    metric = "neighbors"        # metric for dropping dissimilar edges (neighbors, jaccard, cosine)
    object = "links"            # object for defense (links, features)

    model = TwoPartyCNGCN(dataset=dataset, nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1,
                            device=device)
    model.fit(modified_adj1.copy(), adj2.copy(), features, features, labels, idx_train, threshold, metric=metric, object=object,
                train_iters=200, initialize=True, verbose=False, idx_val=idx_val)
    model.eval()
    accuracies = model.test(idx_test)  #accuracy of the model after the defense on all the test data - (all the nodes)
    accuracy_test_attack_3 = model.test(idx_test_attack)
    accuracy_test_clean_3 = model.test(idx_test_clean)
    print("Test accuracy on attack set: ", accuracy_test_attack_3)
    print("Test accuracy on clean set: ", accuracy_test_clean_3)

    # At the end of each repeat, append the accuracy values to the lists:
    accuracy_test_attack_1_values.append(accuracy_test_attack_1)
    accuracy_test_clean_1_values.append(accuracy_test_clean_1)
    accuracy_test_attack_2_values.append(accuracy_test_attack_2)
    accuracy_test_clean_2_values.append(accuracy_test_clean_2)
    accuracy_test_attack_3_values.append(accuracy_test_attack_3)
    accuracy_test_clean_3_values.append(accuracy_test_clean_3)



####################### Standard deviation values ############################
# Calculate the standard deviation of the accuracy values:
accuracy_test_attack_1_std = np.std(accuracy_test_attack_1_values)
accuracy_test_clean_1_std = np.std(accuracy_test_clean_1_values)
accuracy_test_attack_2_std = np.std(accuracy_test_attack_2_values)
accuracy_test_clean_2_std = np.std(accuracy_test_clean_2_values)
accuracy_test_attack_3_std = np.std(accuracy_test_attack_3_values)
accuracy_test_clean_3_std = np.std(accuracy_test_clean_3_values)

# print("Standard deviation of test accuracy on attack set 1: ", accuracy_test_attack_1_std)
# print("Standard deviation of test accuracy on clean set 1: ", accuracy_test_clean_1_std)
# print("Standard deviation of test accuracy on attack set 2: ", accuracy_test_attack_2_std)
# print("Standard deviation of test accuracy on clean set 2: ", accuracy_test_clean_2_std)
# print("Standard deviation of test accuracy on attack set 3: ", accuracy_test_attack_3_std)
# print("Standard deviation of test accuracy on clean set 3: ", accuracy_test_clean_3_std)

# Save the standard deviation values to a file
with open('variables_std.pkl', 'wb') as f:
    pickle.dump([accuracy_test_attack_1_std, accuracy_test_clean_1_std, 
                 accuracy_test_attack_2_std, accuracy_test_clean_2_std, 
                 accuracy_test_attack_3_std, accuracy_test_clean_3_std], f)


####################### Average accuracy values ############################
# Calculate the average accuracy values:
accuracy_test_attack_1_avg = np.mean(accuracy_test_attack_1_values)
accuracy_test_clean_1_avg = np.mean(accuracy_test_clean_1_values)
accuracy_test_attack_2_avg = np.mean(accuracy_test_attack_2_values)
accuracy_test_clean_2_avg = np.mean(accuracy_test_clean_2_values)
accuracy_test_attack_3_avg = np.mean(accuracy_test_attack_3_values)
accuracy_test_clean_3_avg = np.mean(accuracy_test_clean_3_values)

# print("Average test accuracy on attack set 1: ", accuracy_test_attack_1_avg)
# print("Average test accuracy on clean set 1: ", accuracy_test_clean_1_avg)
# print("Average test accuracy on attack set 2: ", accuracy_test_attack_2_avg)
# print("Average test accuracy on clean set 2: ", accuracy_test_clean_2_avg)
# print("Average test accuracy on attack set 3: ", accuracy_test_attack_3_avg)
# print("Average test accuracy on clean set 3: ", accuracy_test_clean_3_avg)

with open('variables.pkl', 'wb') as f:
    pickle.dump([accuracy_test_attack_1_avg, accuracy_test_clean_1_avg, 
                 accuracy_test_attack_2_avg, accuracy_test_clean_2_avg, 
                 accuracy_test_attack_3_avg, accuracy_test_clean_3_avg], f)
    
##################################
                


In [ ]:
################################# V0 - Plotting ACcuracies #######################################
labels = ['ACCs before attack', 'ACCs before attack', 
          'ACCs after attack', 'ACCs after attack', 
          'ACCs after CryptoGraph', 'ACCs after CryptoGraph']
values = [accuracy_test_attack_1_avg, accuracy_test_clean_1_avg, 
          accuracy_test_attack_2_avg, accuracy_test_clean_2_avg, 
          accuracy_test_attack_3_avg, accuracy_test_clean_3_avg]
std_devs = [accuracy_test_attack_1_std, accuracy_test_clean_1_std, 
            accuracy_test_attack_2_std, accuracy_test_clean_2_std, 
            accuracy_test_attack_3_std, accuracy_test_clean_3_std]

# Define the x-coordinates of the bars and the width of the bars
x = np.arange(len(labels)//2)
width = 0.35

# # Create the bar chart
# bars1 = plt.bar(x - width/2, values[::2], width, label='Attack set', color='red')
# bars2 = plt.bar(x + width/2, values[1::2], width, label='Clean set', color='blue')

# Create the bar chart with error bars
bars1 = plt.bar(x - width/2, values[::2], width, yerr=std_devs[::2], label='Attack set', color='red', capsize=5)
bars2 = plt.bar(x + width/2, values[1::2], width, yerr=std_devs[1::2], label='Clean set', color='blue', capsize=5)

# Add a title and labels to the axes
plt.title('Average Accuracies')
plt.xlabel('Levels of Attack and Defense')
plt.ylabel('Average Accuracy')

# Make the x-labels vertical
plt.xticks(x, labels[::2])

# Add a legend
plt.legend()

# Add exact values on each bar
for bars in [bars1, bars2]:
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Adjust the layout to prevent the labels from being cut off
plt.tight_layout()

# Display the chart
plt.show()

In [ ]:
################################# Attack one node (adding label) each time to discover node conditions ###############################
# Define the number of repeats
num_repeats = 1
accuracy_test_attack_1_values = []
accuracy_test_clean_1_values = []
accuracy_test_attack_2_values = []
accuracy_test_clean_2_values = []
accuracy_test_attack_3_values = []
accuracy_test_clean_3_values = []

################################ Data loading #######################################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load data
dataset = "polblogs"
# Use the current directory for windows
data = Dataset(root='.', name=dataset)
#data = Dataset(root='/tmp/', name=dataset): this is for unix-based systems

for _ in range(num_repeats):
    
    #############################  preprocessing ############################
    adj, features, labels = data.adj, data.features, data.labels
    idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

    #split idx_test into two parts : one node for attack and the rest for clean
    # test_size = 1  # 1 node for attack others for test_clean
    # idx_test_clean , idx_test_attack = train_test_split(idx_test, test_size=test_size, random_state=42)
    # selected_node = idx_test_attack[0]
    # print(f"Selected node for attack: {selected_node}")

    #choose a node for attack manually (the one which has been chosen randomely from idx_test in the previous part)
    selected_node = 1196
    idx_test_attack = np.array([selected_node])
    idx_test_clean = np.array([node for node in idx_test if node != selected_node])

    # Split graph into two graphs for Crypto'Graph
    proportion_of_common_links = 0.5
    adj1, adj2 = split_dataset(adj, proportion_of_common_links) 

   
    # print ("idx_test_clean",len (idx_test_clean))
    # print ("idx_test_attack",len (idx_test_attack))
    # print ("idx_test",len (idx_test))
    # print ("train",len (idx_train))
    # print ("val",len (idx_val))

    # print ("idx_test_clean",idx_test_clean)
    # print ("idx_test_attack",idx_test_attack)

    ############################ tarin model initially and test accuracy ###########################
    # Perform evaluation before attack to find the baseline accuracy

    # Train GCN model 
    # 1. Model definition
    model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device, dropout=0.5)
    model = model.to(device)
    # 2. Train model - patience= number of epochs to wait before early stopping , train_iters= number of iterations
    model.fit(features, adj, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200)
    #model.fit(features, adj, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200,metric=metric,
               #object=object, initialize=True, verbose=False )
    # 3. Evaluate model
    model.eval()
    output = model.test(idx_test)
    #acc_test = accuracy(output, labels, idx_test)

    # accuracies = model.test(idx_test) 
    # print("Test accuracy: ", accuracies)

    # output_1 = model.test(idx_test_attack)
    # output_2 = model.test(idx_test_clean)

    accuracy_test_attack_1 = model.test(idx_test_attack) 
    accuracy_test_clean_1 = model.test(idx_test_clean)

    print("Test accuracy on attack set: ", accuracy_test_attack_1)
    print("Test accuracy on clean set: ", accuracy_test_clean_1)

    
    ################################ Mahsa attack ##############################
    # Perform the attack
    # version 0: attack on the whole graph - on idx_test_attack nodes

    modified_adj1 =  adj1.copy()

    # Create a NetworkX graph from the adjacency matrix
    graph = nx.from_scipy_sparse_array(modified_adj1)

    # Add labels to the graph
    for node_id, label in enumerate(labels):
        graph.nodes[node_id]['label'] = label
    # print(f"lenght of the labels : {len(labels)}") 
    print(f"graph edges : {graph.number_of_edges()}")
    print(f"graph nodes : {graph.number_of_nodes()}")
    # Set the budget for the attack
    budget = 50

    #attacked_graph is initially set to graph, and then updated after each attack
    # This means that each attack is performed on the graph resulting from the previous attacks.
    attacked_graph = graph
    for target_node in idx_test_attack:
        non_neighbor_opposit = backdoor.find_non_neighbor_opposit_label(attacked_graph, target_node)
        max_same_min_opposit_label_neighbors = backdoor.find_max_same_min_opposit_label_neighbors(attacked_graph, non_neighbor_opposit)
        nodes_for_attack = backdoor.nodes_for_attack(attacked_graph, target_node, max_same_min_opposit_label_neighbors, budget)
        attacked_graph = backdoor.insert_edge(attacked_graph, target_node, nodes_for_attack)
        backdoor.evaluate_graph(attacked_graph, nodes_for_attack, target_node, budget)
   
    print ( f"idx_test_attack: {idx_test_attack}")
    neighbors_same_label = backdoor.find_same_neighbor(graph, selected_node)
    neighbors_opposit_label = backdoor.find_opposit_neighbor(graph, selected_node)
    print ( f"number of target same label neighbors: {len(neighbors_same_label)}")
    print ( f"number of target opposit label neighbors: {len(neighbors_opposit_label)}")
    print ( f"idx_test_attack same label neighbors: {neighbors_same_label}")
    print ( f"idx_test_attack opposit label neighbors: {neighbors_opposit_label}")
    print ( f"number of target NONE neighbors opposite label: {len(backdoor.find_non_neighbor_opposit_label(graph, selected_node))}")
    print ( f"idx_test_attack opposit label NONE neighbors: {backdoor.find_non_neighbor_opposit_label(graph, selected_node)}")

    ################################## Plotting target node before attack  #######################################
    target_subgraph = graph.subgraph([selected_node] + list(neighbors_same_label) + list(neighbors_opposit_label))
    pos = nx.spring_layout(target_subgraph)  # positions for all nodes
    # nodes
    nx.draw_networkx_nodes(target_subgraph, pos, nodelist=neighbors_same_label, node_color='green')
    nx.draw_networkx_nodes(target_subgraph, pos, nodelist=neighbors_opposit_label, node_color='pink')
    # edges
    nx.draw_networkx_edges(target_subgraph, pos, width=1.0, alpha=0.5)
    # labels
    nx.draw_networkx_labels(target_subgraph, pos, font_size=16)
    plt.axis('off')
    plt.show()

   
    # Convert the graph to a CSR matrix
    modified_adj1 =backdoor.convert(attacked_graph)
    print(attacked_graph)
    print(f"total inserted edges : {(attacked_graph.number_of_edges() - graph.number_of_edges())}")

    ################################ evaluation after attack ##############################
    # accuracy after attack
    model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device, dropout=0.5)
    model = model.to(device)
    #data = data.to(device)
    model.fit(features, modified_adj1, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200)
    model.eval()

    # output = model.test(idx_test)

    #acc_test = accuracy(output, labels, idx_test)

    # output_1 = model.test(idx_test_attack)
    # output_2 = model.test(idx_test_clean)

    accuracy_test_attack_2 = model.test(idx_test_attack) 
    accuracy_test_clean_2 = model.test(idx_test_clean)

    print("Test accuracy on attack set after attack: ", accuracy_test_attack_2)
    print("Test accuracy on clean set after attack: ", accuracy_test_clean_2)

    ############################ Crypto'Graph defense ###########################
    # Perform Crypto'Graph distributed defense

    threshold = 2               # threshold for dropping dissimilar edges
    metric = "neighbors"        # metric for dropping dissimilar edges (neighbors, jaccard, cosine)
    object = "links"            # object for defense (links, features)

    model = TwoPartyCNGCN(dataset=dataset, nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1,
                            device=device)
    model.fit(modified_adj1.copy(), adj2.copy(), features, features, labels, idx_train, threshold, metric=metric, object=object,
                train_iters=200, initialize=True, verbose=False, idx_val=idx_val)
    model.eval()
    accuracies = model.test(idx_test)  #accuracy of the model after the defense on all the test data - (all the nodes)
    accuracy_test_attack_3 = model.test(idx_test_attack)
    accuracy_test_clean_3 = model.test(idx_test_clean)
    print("Test accuracy on attack set after Crypto'Graph: ", accuracy_test_attack_3)
    print("Test accuracy on clean set after Crypto'Graph: ", accuracy_test_clean_3)

    # At the end of each repeat, append the accuracy values to the lists:
    accuracy_test_attack_1_values.append(accuracy_test_attack_1)
    accuracy_test_clean_1_values.append(accuracy_test_clean_1)
    accuracy_test_attack_2_values.append(accuracy_test_attack_2)
    accuracy_test_clean_2_values.append(accuracy_test_clean_2)
    accuracy_test_attack_3_values.append(accuracy_test_attack_3)
    accuracy_test_clean_3_values.append(accuracy_test_clean_3)



####################### Standard deviation values ############################
# Calculate the standard deviation of the accuracy values:
accuracy_test_attack_1_std = np.std(accuracy_test_attack_1_values)
accuracy_test_clean_1_std = np.std(accuracy_test_clean_1_values)
accuracy_test_attack_2_std = np.std(accuracy_test_attack_2_values)
accuracy_test_clean_2_std = np.std(accuracy_test_clean_2_values)
accuracy_test_attack_3_std = np.std(accuracy_test_attack_3_values)
accuracy_test_clean_3_std = np.std(accuracy_test_clean_3_values)

# print("Standard deviation of test accuracy on attack set 1: ", accuracy_test_attack_1_std)
# print("Standard deviation of test accuracy on clean set 1: ", accuracy_test_clean_1_std)
# print("Standard deviation of test accuracy on attack set 2: ", accuracy_test_attack_2_std)
# print("Standard deviation of test accuracy on clean set 2: ", accuracy_test_clean_2_std)
# print("Standard deviation of test accuracy on attack set 3: ", accuracy_test_attack_3_std)
# print("Standard deviation of test accuracy on clean set 3: ", accuracy_test_clean_3_std)

# Save the standard deviation values to a file
with open('variables_std.pkl', 'wb') as f:
    pickle.dump([accuracy_test_attack_1_std, accuracy_test_clean_1_std, 
                 accuracy_test_attack_2_std, accuracy_test_clean_2_std, 
                 accuracy_test_attack_3_std, accuracy_test_clean_3_std], f)


####################### Average accuracy values ############################
# Calculate the average accuracy values:
accuracy_test_attack_1_avg = np.mean(accuracy_test_attack_1_values)
accuracy_test_clean_1_avg = np.mean(accuracy_test_clean_1_values)
accuracy_test_attack_2_avg = np.mean(accuracy_test_attack_2_values)
accuracy_test_clean_2_avg = np.mean(accuracy_test_clean_2_values)
accuracy_test_attack_3_avg = np.mean(accuracy_test_attack_3_values)
accuracy_test_clean_3_avg = np.mean(accuracy_test_clean_3_values)


with open('variables.pkl', 'wb') as f:
    pickle.dump([accuracy_test_attack_1_avg, accuracy_test_clean_1_avg, 
                 accuracy_test_attack_2_avg, accuracy_test_clean_2_avg, 
                 accuracy_test_attack_3_avg, accuracy_test_clean_3_avg], f)
    

################################# Plotting Training Results#######################################
labels = ['ACCs before attack', 'ACCs before attack', 
          'ACCs after attack', 'ACCs after attack', 
          'ACCs after CryptoGraph', 'ACCs after CryptoGraph']
values = [accuracy_test_attack_1_avg, accuracy_test_clean_1_avg, 
          accuracy_test_attack_2_avg, accuracy_test_clean_2_avg, 
          accuracy_test_attack_3_avg, accuracy_test_clean_3_avg]
std_devs = [accuracy_test_attack_1_std, accuracy_test_clean_1_std, 
            accuracy_test_attack_2_std, accuracy_test_clean_2_std, 
            accuracy_test_attack_3_std, accuracy_test_clean_3_std]

# Define the x-coordinates of the bars and the width of the bars
x = np.arange(len(labels)//2)
width = 0.35

# Create the bar chart with error bars
bars1 = plt.bar(x - width/2, values[::2], width, yerr=std_devs[::2], label='Attack set', color='red', capsize=5)
bars2 = plt.bar(x + width/2, values[1::2], width, yerr=std_devs[1::2], label='Clean set', color='blue', capsize=5)

# Add a title and labels to the axes
plt.title('Average Accuracies')
plt.xlabel('Levels of Attack and Defense')
plt.ylabel('Average Accuracy')

# Make the x-labels vertical
plt.xticks(x, labels[::2])
# Add a legend
plt.legend()
# Add exact values on each bar
for bars in [bars1, bars2]:
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Adjust the layout to prevent the labels from being cut off
plt.tight_layout()
# Display the chart
plt.show()                

In [ ]:
################################# V1: Attack by removing edges ###############################
################################# V2: Attack by removing edges + consider cummon neighbors for Crypto ###############################

# Define the number of repeats
num_repeats = 1
accuracy_test_attack_1_values = []
accuracy_test_clean_1_values = []
accuracy_test_attack_2_values = []
accuracy_test_clean_2_values = []
accuracy_test_attack_3_values = []
accuracy_test_clean_3_values = []

################################ Data loading #######################################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load data
dataset = "polblogs"
# Use the current directory for windows
data = Dataset(root='.', name=dataset)
#data = Dataset(root='/tmp/', name=dataset): this is for unix-based systems

for _ in range(num_repeats):
    
    #############################  preprocessing ############################
    adj, features, labels = data.adj, data.features, data.labels
    idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

    #split idx_test into two parts : one node for attack and the rest for clean
    # test_size = 1  # 1 node for attack others for test_clean
    # idx_test_clean , idx_test_attack = train_test_split(idx_test, test_size=test_size, random_state=42)
    # selected_node = idx_test_attack[0]
    # print(f"Selected node for attack: {selected_node}")

    #choose a node for attack manually (the one which has been chosen randomely from idx_test in the previous part)
    selected_node = 1196
    idx_test_attack = np.array([selected_node])
    idx_test_clean = np.array([node for node in idx_test if node != selected_node])

    # Split graph into two graphs for Crypto'Graph
    proportion_of_common_links = 0.5
    adj1, adj2 = split_dataset(adj, proportion_of_common_links) 

   
    # print ("idx_test_clean",len (idx_test_clean))
    # print ("idx_test_attack",len (idx_test_attack))
    # print ("idx_test",len (idx_test))
    # print ("train",len (idx_train))
    # print ("val",len (idx_val))
    # print ("idx_test_clean",idx_test_clean)
    # print ("idx_test_attack",idx_test_attack)

    ############################ tarin model initially and test accuracy ###########################
    # Perform evaluation before attack to find the baseline accuracy

    # Train GCN model 
    # 1. Model definition
    model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device, dropout=0.5)
    model = model.to(device)
    # 2. Train model - patience= number of epochs to wait before early stopping , train_iters= number of iterations
    model.fit(features, adj, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200)
    #model.fit(features, adj, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200,metric=metric,
               #object=object, initialize=True, verbose=False )
    # 3. Evaluate model
    model.eval()
    output = model.test(idx_test)
    #acc_test = accuracy(output, labels, idx_test)
    # accuracies = model.test(idx_test) 
    # print("Test accuracy: ", accuracies)
    # output_1 = model.test(idx_test_attack)
    # output_2 = model.test(idx_test_clean)

    accuracy_test_attack_1 = model.test(idx_test_attack) 
    accuracy_test_clean_1 = model.test(idx_test_clean)
    print("Test accuracy on attack set: ", accuracy_test_attack_1)
    print("Test accuracy on clean set: ", accuracy_test_clean_1)

    
    ################################ Mahsa attack ##############################
    # Perform the attack
    # version 1: attack on the whole graph - on idx_test_attack nodes - by only removing edges

    modified_adj1 =  adj1.copy()

    # Create a NetworkX graph from the adjacency matrix
    graph = nx.from_scipy_sparse_array(modified_adj1)
    # Add labels to the graph
    for node_id, label in enumerate(labels):
        graph.nodes[node_id]['label'] = label
    # print(f"lenght of the labels : {len(labels)}") 
    print(f"graph edges : {graph.number_of_edges()}")
    print(f"graph nodes : {graph.number_of_nodes()}")
    
    # Set the budget for the attack
    budget = 50

    #attacked_graph is initially set to graph, and then updated after each attack
    # This means that each attack is performed on the graph resulting from the previous attacks.
    attacked_graph = graph
    for target_node in idx_test_attack:
        neighbor_same = backdoor.find_same_neighbor(attacked_graph, target_node)
        max_same_min_opposit_label_neighbors = backdoor.find_max_same_min_opposit_label_neighbors(attacked_graph, neighbor_same)
        # step1: we dont consider common neighbors
        # attacked_graph = backdoor.remove_edge1(attacked_graph, target_node, max_same_min_opposit_label_neighbors, budget)
        
        # step2: we consider common neighbors
        nodes_for_remove = backdoor.nodes_for_remove(attacked_graph, target_node, max_same_min_opposit_label_neighbors, budget)
        attacked_graph = backdoor.remove_edge2(attacked_graph, target_node, nodes_for_remove)
        backdoor.check_removing(attacked_graph,nodes_for_remove , target_node, budget)
   
    print ( f"idx_test_attack: {idx_test_attack}")
    neighbors_same_label = backdoor.find_same_neighbor(graph, selected_node)
    neighbors_opposit_label = backdoor.find_opposit_neighbor(graph, selected_node)
    print ( f"number of target same label neighbors: {len(neighbors_same_label)}")
    print ( f"number of target opposit label neighbors: {len(neighbors_opposit_label)}")
    print ( f"idx_test_attack same label neighbors: {neighbors_same_label}")
    print ( f"idx_test_attack opposit label neighbors: {neighbors_opposit_label}")
    print ( f"number of target NONE neighbors opposite label: {len(backdoor.find_non_neighbor_opposit_label(graph, selected_node))}")
    print ( f"idx_test_attack opposit label NONE neighbors: {backdoor.find_non_neighbor_opposit_label(graph, selected_node)}")

    
    ################################## Plotting target node before attack  #######################################
    target_subgraph = graph.subgraph([selected_node] + list(neighbors_same_label) + list(neighbors_opposit_label))
    pos = nx.spring_layout(target_subgraph)  # positions for all nodes
    # nodes
    nx.draw_networkx_nodes(target_subgraph, pos, nodelist=neighbors_same_label, node_color='green')
    nx.draw_networkx_nodes(target_subgraph, pos, nodelist=neighbors_opposit_label, node_color='pink')
    # edges
    nx.draw_networkx_edges(target_subgraph, pos, width=1.0, alpha=0.5)
    # labels
    nx.draw_networkx_labels(target_subgraph, pos, font_size=16)
    plt.axis('off')
    plt.show()

   
    # Convert the graph to a CSR matrix
    modified_adj1 =backdoor.convert(attacked_graph)
    print(attacked_graph)
    print(f"total removed edges : {(graph.number_of_edges() - attacked_graph.number_of_edges())}")

    ################################ evaluation after attack ##############################
    # accuracy after attack
    model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device, dropout=0.5)
    model = model.to(device)
    #data = data.to(device)
    model.fit(features, modified_adj1, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200)
    model.eval()

    # output = model.test(idx_test)

    #acc_test = accuracy(output, labels, idx_test)

    # output_1 = model.test(idx_test_attack)
    # output_2 = model.test(idx_test_clean)

    accuracy_test_attack_2 = model.test(idx_test_attack) 
    accuracy_test_clean_2 = model.test(idx_test_clean)

    print("Test accuracy on attack set after attack: ", accuracy_test_attack_2)
    print("Test accuracy on clean set after attack: ", accuracy_test_clean_2)

    ############################ Crypto'Graph defense ###########################
    # Perform Crypto'Graph distributed defense

    threshold = 2               # threshold for dropping dissimilar edges
    metric = "neighbors"        # metric for dropping dissimilar edges (neighbors, jaccard, cosine)
    object = "links"            # object for defense (links, features)

    model = TwoPartyCNGCN(dataset=dataset, nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1,
                            device=device)
    model.fit(modified_adj1.copy(), adj2.copy(), features, features, labels, idx_train, threshold, metric=metric, object=object,
                train_iters=200, initialize=True, verbose=False, idx_val=idx_val)
    model.eval()
    accuracies = model.test(idx_test)  #accuracy of the model after the defense on all the test data - (all the nodes)
    accuracy_test_attack_3 = model.test(idx_test_attack)
    accuracy_test_clean_3 = model.test(idx_test_clean)
    print("Test accuracy on attack set after Crypto'Graph: ", accuracy_test_attack_3)
    print("Test accuracy on clean set after Crypto'Graph: ", accuracy_test_clean_3)

    # At the end of each repeat, append the accuracy values to the lists:
    accuracy_test_attack_1_values.append(accuracy_test_attack_1)
    accuracy_test_clean_1_values.append(accuracy_test_clean_1)
    accuracy_test_attack_2_values.append(accuracy_test_attack_2)
    accuracy_test_clean_2_values.append(accuracy_test_clean_2)
    accuracy_test_attack_3_values.append(accuracy_test_attack_3)
    accuracy_test_clean_3_values.append(accuracy_test_clean_3)



####################### Standard deviation values ############################
# Calculate the standard deviation of the accuracy values:
accuracy_test_attack_1_std = np.std(accuracy_test_attack_1_values)
accuracy_test_clean_1_std = np.std(accuracy_test_clean_1_values)
accuracy_test_attack_2_std = np.std(accuracy_test_attack_2_values)
accuracy_test_clean_2_std = np.std(accuracy_test_clean_2_values)
accuracy_test_attack_3_std = np.std(accuracy_test_attack_3_values)
accuracy_test_clean_3_std = np.std(accuracy_test_clean_3_values)

# print("Standard deviation of test accuracy on attack set 1: ", accuracy_test_attack_1_std)
# print("Standard deviation of test accuracy on clean set 1: ", accuracy_test_clean_1_std)
# print("Standard deviation of test accuracy on attack set 2: ", accuracy_test_attack_2_std)
# print("Standard deviation of test accuracy on clean set 2: ", accuracy_test_clean_2_std)
# print("Standard deviation of test accuracy on attack set 3: ", accuracy_test_attack_3_std)
# print("Standard deviation of test accuracy on clean set 3: ", accuracy_test_clean_3_std)

# Save the standard deviation values to a file
with open('variables_std.pkl', 'wb') as f:
    pickle.dump([accuracy_test_attack_1_std, accuracy_test_clean_1_std, 
                 accuracy_test_attack_2_std, accuracy_test_clean_2_std, 
                 accuracy_test_attack_3_std, accuracy_test_clean_3_std], f)


####################### Average accuracy values ############################
# Calculate the average accuracy values:
accuracy_test_attack_1_avg = np.mean(accuracy_test_attack_1_values)
accuracy_test_clean_1_avg = np.mean(accuracy_test_clean_1_values)
accuracy_test_attack_2_avg = np.mean(accuracy_test_attack_2_values)
accuracy_test_clean_2_avg = np.mean(accuracy_test_clean_2_values)
accuracy_test_attack_3_avg = np.mean(accuracy_test_attack_3_values)
accuracy_test_clean_3_avg = np.mean(accuracy_test_clean_3_values)


with open('variables.pkl', 'wb') as f:
    pickle.dump([accuracy_test_attack_1_avg, accuracy_test_clean_1_avg, 
                 accuracy_test_attack_2_avg, accuracy_test_clean_2_avg, 
                 accuracy_test_attack_3_avg, accuracy_test_clean_3_avg], f)
    

################################# Plotting Training Results #######################################
labels = ['ACCs before attack', 'ACCs before attack', 
          'ACCs after attack', 'ACCs after attack', 
          'ACCs after CryptoGraph', 'ACCs after CryptoGraph']
values = [accuracy_test_attack_1_avg, accuracy_test_clean_1_avg, 
          accuracy_test_attack_2_avg, accuracy_test_clean_2_avg, 
          accuracy_test_attack_3_avg, accuracy_test_clean_3_avg]
std_devs = [accuracy_test_attack_1_std, accuracy_test_clean_1_std, 
            accuracy_test_attack_2_std, accuracy_test_clean_2_std, 
            accuracy_test_attack_3_std, accuracy_test_clean_3_std]

# Define the x-coordinates of the bars and the width of the bars
x = np.arange(len(labels)//2)
width = 0.35

# Create the bar chart with error bars
bars1 = plt.bar(x - width/2, values[::2], width, yerr=std_devs[::2], label='Attack set', color='red', capsize=5)
bars2 = plt.bar(x + width/2, values[1::2], width, yerr=std_devs[1::2], label='Clean set', color='blue', capsize=5)

# Add a title and labels to the axes
plt.title('Average Accuracies')
plt.xlabel('Levels of Attack and Defense')
plt.ylabel('Average Accuracy')

# Make the x-labels vertical
plt.xticks(x, labels[::2])
# Add a legend
plt.legend()
# Add exact values on each bar
for bars in [bars1, bars2]:
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Adjust the layout to prevent the labels from being cut off
plt.tight_layout()
# Display the chart
plt.show()                

In [ ]:
################# TESTTTTTTT for analysis of the graph and nodes before attack ###############################

################################ Data loading #######################################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load data
dataset = "polblogs"
# Use the current directory for windows
data = Dataset(root='.', name=dataset)
#data = Dataset(root='/tmp/', name=dataset): this is for unix-based systems

#############################  preprocessing ############################
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

modified_adj =  adj.copy()

# Create a NetworkX graph from the adjacency matrix
graph = nx.from_scipy_sparse_array(modified_adj)
# Add labels to the graph
for node_id, label in enumerate(labels):
    graph.nodes[node_id]['label'] = label
# print(f"lenght of the labels : {len(labels)}") 
print(f"graph edges : {graph.number_of_edges()}")
print(f"graph nodes : {graph.number_of_nodes()}")

#########################analysis of the graph and nodes before attack ###############################

# all 
def count_neighbor_labels(graph, labels):
    count_dict = {} # dictionary type variable with key: node, and value of: (same_label, opposite_label)
    for node in graph.nodes():
        neighbors = list(graph.neighbors(node))
        same_label = sum(labels[neighbor] == labels[node] for neighbor in neighbors)
        opposite_label = len(neighbors) - same_label
        count_dict[node] = (same_label, opposite_label)
    return count_dict

count_dict = count_neighbor_labels(graph, labels)
# Sort nodes by the number of same-label neighbors. 
# x is tuple (node, (same_label, opposite_label)). so x[1][0] is index 0 from the second element of the tuple = same_lable 
sorted_nodes = sorted(count_dict.items(), key=lambda x: x[1][0], reverse=True) 

print(f"Nodes and their same-label and opposite-label neighbors: {count_dict}")

# returns a dict of nodes which have most same label and most opposite labels with the target node
def sort_classly(count_dict, labels, target_node):
    nodes_class_same = {node: counts for node, counts in count_dict.items() if labels[node] == labels[target_node]}
    nodes_class_opp = {node: counts for node, counts in count_dict.items() if labels[node] != labels[target_node]}
    sorted_nodes_same = sorted(nodes_class_same.items(), key=lambda x: x[1][0], reverse=True)
    sorted_nodes_opp = sorted(nodes_class_opp.items(), key=lambda x: x[1][0], reverse=True)
    return sorted_nodes_same, sorted_nodes_opp

sorted_nodes_same, sorted_nodes_opp = sort_classly(count_dict, labels, 499)
# Print the top 10 nodes for each class
print(f"The most same-label neighbors in same class of label with node {499}: {sorted_nodes_same[:10]}")
print(f"The most same-label neighbors in class 1: {sorted_nodes_opp[:10]}")
print (f"label of node is: {labels[671]}")

In [ ]:
################################# V3: Attack by removing edges + adding edge ###############################
################### Attack by removing edges + adding edges: for 10% nodes, for one node, for manually selection of nodes 
# Define the number of repeats
num_repeats = 10
accuracy_test_attack_1_values = []
accuracy_test_clean_1_values = []
accuracy_test_attack_2_values = []
accuracy_test_clean_2_values = []
accuracy_test_attack_3_values = []
accuracy_test_clean_3_values = []

################################ Data loading #######################################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load data
dataset = "polblogs"
# Use the current directory for windows
data = Dataset(root='.', name=dataset)
#data = Dataset(root='/tmp/', name=dataset): this is for unix-based systems

for _ in range(num_repeats):
    
    #############################  preprocessing ############################
    adj, features, labels = data.adj, data.features, data.labels
    idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

    ################### split idx_test into two parts : one node for attack randomley and the rest for clean
    # test_size = 1  # 1 node for attack others for test_clean
    # idx_test_clean , idx_test_attack = train_test_split(idx_test, test_size=test_size, random_state=42)
    # selected_node = idx_test_attack[0]
    # print(f"Selected node for attack: {selected_node}")
    ######################

    ############### choose a node for attack manually (the one which has been chosen randomely from idx_test in the previous part)
    #selected_node = 1196
    # idx_test_attack = np.array([selected_node])
    # idx_test_clean = np.array([node for node in idx_test if node != selected_node])
    ######################

    ################ split idx_test into two parts randomly with a specific ratio %
    test_size = 0.10  # 10% for test_attack, 90% for test_clean
    idx_test_clean , idx_test_attack = train_test_split(idx_test, test_size=test_size, random_state=42)
    ######################

    # Split graph into two graphs for Crypto'Graph
    proportion_of_common_links = 0.5
    adj1, adj2 = split_dataset(adj, proportion_of_common_links) 

   
    # print ("idx_test_clean",len (idx_test_clean))
    # print ("idx_test_attack",len (idx_test_attack))
    # print ("idx_test",len (idx_test))
    # print ("train",len (idx_train))
    # print ("val",len (idx_val))
    # print ("idx_test_clean",idx_test_clean)
    # print ("idx_test_attack",idx_test_attack)

    ############################ tarin model initially and test accuracy ###########################
    # Perform evaluation before attack to find the baseline accuracy

    # Train GCN model 
    # 1. Model definition
    model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device, dropout=0.5)
    model = model.to(device)
    # 2. Train model - patience= number of epochs to wait before early stopping , train_iters= number of iterations
    model.fit(features, adj, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200)
    #model.fit(features, adj, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200,metric=metric,
               #object=object, initialize=True, verbose=False )
    # 3. Evaluate model
    model.eval()
    output = model.test(idx_test)

    accuracy_test_attack_1 = model.test(idx_test_attack) 
    accuracy_test_clean_1 = model.test(idx_test_clean)
    print("Test accuracy on attack set: ", accuracy_test_attack_1)
    print("Test accuracy on clean set: ", accuracy_test_clean_1)

    
    ################################ Mahsa attack ##############################
    # Perform the attack
    # version 1: attack on the whole graph - on idx_test_attack nodes - by only removing edges

    modified_adj1 =  adj1.copy()

    # Create a NetworkX graph from the adjacency matrix
    graph = nx.from_scipy_sparse_array(modified_adj1)
    # Add labels to the graph
    for node_id, label in enumerate(labels):
        graph.nodes[node_id]['label'] = label
    # print(f"lenght of the labels : {len(labels)}") 
    print(f"graph edges : {graph.number_of_edges()}")
    print(f"graph nodes : {graph.number_of_nodes()}")
    
    # Set the budget for the attack
    budget = 5

    #attacked_graph is initially set to graph, and then updated after each attack
    # This means that each attack is performed on the graph resulting from the previous attacks.
    attacked_graph = graph
    count_dict = backdoor.count_neighbor_labels(graph, labels)
    for target_node in idx_test_attack:
        sorted_nodes_same, sorted_nodes_opp = backdoor.sort_classly(count_dict, labels, target_node)# sort nodes based on the number of same and opposite label neighbors
        remove_budget = budget // 2
        add_budget = budget - remove_budget
        attacked_graph,removed_edges,added_edges = backdoor.add_remove(attacked_graph, target_node, sorted_nodes_same, sorted_nodes_opp, remove_budget, add_budget)
            
               
    print(f"Removed {len(removed_edges)} edges: {removed_edges}")
    print(f"Added {len(added_edges)} edges: {added_edges}")    
    
    print ( f"idx_test_attack: {idx_test_attack}")
    neighbors_same_label = backdoor.find_same_neighbor(graph, selected_node)
    neighbors_opposit_label = backdoor.find_opposit_neighbor(graph, selected_node)
    print ( f"number of target same label neighbors before attack: {len(neighbors_same_label)}")
    print ( f"number of target opposit label neighbors before attack: {len(neighbors_opposit_label)}")
    print( f"idx_test_attack same all over the graph : {len(sorted_nodes_same)}")
    print( f"idx_test_attack opposit label all over the graph: {len(sorted_nodes_opp)}")
    
    neighbors_same_after = backdoor.find_same_neighbor(attacked_graph, selected_node)
    neighbors_opposit_after = backdoor.find_opposit_neighbor(attacked_graph, selected_node)
    print ( f"idx_test_attack same label neighbors after attack: {len(neighbors_same_after)}")
    print ( f"idx_test_attack opposit label neighbors after attack: {len(neighbors_opposit_after)}")


    print ( f"idx_test_attack same label neighbors: {neighbors_same_label}")
    print ( f"idx_test_attack opposit label neighbors: {neighbors_opposit_label}")
    print (f"same label all over graph : (node, (num of same label, num of opposit label)){sorted_nodes_same}")
    print (f"as above for opposite lables{sorted_nodes_opp}")
    
    def is_neighbor(graph, node1, node2):
        return graph.has_edge(node1, node2) or graph.has_edge(node2, node1)

    is_neighbor = is_neighbor(attacked_graph, 300, 499)
    print ( f"label of target node is: {labels[499]}")
    print ( f"label of node 300 is: {labels[300]}")
    print ( is_neighbor)

    ################################## Plotting target node before attack  #######################################
    target_subgraph = graph.subgraph([selected_node] + list(neighbors_same_label) + list(neighbors_opposit_label))
    pos = nx.spring_layout(target_subgraph)  # positions for all nodes
    # nodes
    nx.draw_networkx_nodes(target_subgraph, pos, nodelist=neighbors_same_label, node_color='green')
    nx.draw_networkx_nodes(target_subgraph, pos, nodelist=neighbors_opposit_label, node_color='pink')
    # edges
    nx.draw_networkx_edges(target_subgraph, pos, width=1.0, alpha=0.5)
    # labels
    nx.draw_networkx_labels(target_subgraph, pos, font_size=16)
    plt.axis('off')
    plt.show()

   
    # Convert the graph to a CSR matrix
    modified_adj1 =backdoor.convert(attacked_graph)
    print(attacked_graph)
    #print(f"total removed edges : {(graph.number_of_edges() - attacked_graph.number_of_edges())}")

    ################################ evaluation after attack ##############################
    # accuracy after attack
    model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device, dropout=0.5)
    model = model.to(device)
    #data = data.to(device)
    model.fit(features, modified_adj1, labels, idx_train, idx_val=idx_val, patience=30, train_iters=200)
    model.eval()

    # output = model.test(idx_test)

    #acc_test = accuracy(output, labels, idx_test)

    # output_1 = model.test(idx_test_attack)
    # output_2 = model.test(idx_test_clean)

    accuracy_test_attack_2 = model.test(idx_test_attack) 
    accuracy_test_clean_2 = model.test(idx_test_clean)

    print("Test accuracy on attack set after attack: ", accuracy_test_attack_2)
    print("Test accuracy on clean set after attack: ", accuracy_test_clean_2)

    ############################ Crypto'Graph defense ###########################
    # Perform Crypto'Graph distributed defense

    threshold = 2               # threshold for dropping dissimilar edges
    metric = "neighbors"        # metric for dropping dissimilar edges (neighbors, jaccard, cosine)
    object = "links"            # object for defense (links, features)

    model = TwoPartyCNGCN(dataset=dataset, nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1,
                            device=device)
    model.fit(modified_adj1.copy(), adj2.copy(), features, features, labels, idx_train, threshold, metric=metric, object=object,
                train_iters=200, initialize=True, verbose=False, idx_val=idx_val)
    model.eval()
    accuracies = model.test(idx_test)  #accuracy of the model after the defense on all the test data - (all the nodes)
    accuracy_test_attack_3 = model.test(idx_test_attack)
    accuracy_test_clean_3 = model.test(idx_test_clean)
    print("Test accuracy on attack set after Crypto'Graph: ", accuracy_test_attack_3)
    print("Test accuracy on clean set after Crypto'Graph: ", accuracy_test_clean_3)

    # At the end of each repeat, append the accuracy values to the lists:
    accuracy_test_attack_1_values.append(accuracy_test_attack_1)
    accuracy_test_clean_1_values.append(accuracy_test_clean_1)
    accuracy_test_attack_2_values.append(accuracy_test_attack_2)
    accuracy_test_clean_2_values.append(accuracy_test_clean_2)
    accuracy_test_attack_3_values.append(accuracy_test_attack_3)
    accuracy_test_clean_3_values.append(accuracy_test_clean_3)



####################### Standard deviation values ############################
# Calculate the standard deviation of the accuracy values:
accuracy_test_attack_1_std = np.std(accuracy_test_attack_1_values)
accuracy_test_clean_1_std = np.std(accuracy_test_clean_1_values)
accuracy_test_attack_2_std = np.std(accuracy_test_attack_2_values)
accuracy_test_clean_2_std = np.std(accuracy_test_clean_2_values)
accuracy_test_attack_3_std = np.std(accuracy_test_attack_3_values)
accuracy_test_clean_3_std = np.std(accuracy_test_clean_3_values)

# Save the standard deviation values to a file
with open('variables_std.pkl', 'wb') as f:
    pickle.dump([accuracy_test_attack_1_std, accuracy_test_clean_1_std, 
                 accuracy_test_attack_2_std, accuracy_test_clean_2_std, 
                 accuracy_test_attack_3_std, accuracy_test_clean_3_std], f)


####################### Average accuracy values ############################
# Calculate the average accuracy values:
accuracy_test_attack_1_avg = np.mean(accuracy_test_attack_1_values)
accuracy_test_clean_1_avg = np.mean(accuracy_test_clean_1_values)
accuracy_test_attack_2_avg = np.mean(accuracy_test_attack_2_values)
accuracy_test_clean_2_avg = np.mean(accuracy_test_clean_2_values)
accuracy_test_attack_3_avg = np.mean(accuracy_test_attack_3_values)
accuracy_test_clean_3_avg = np.mean(accuracy_test_clean_3_values)


with open('variables.pkl', 'wb') as f:
    pickle.dump([accuracy_test_attack_1_avg, accuracy_test_clean_1_avg, 
                 accuracy_test_attack_2_avg, accuracy_test_clean_2_avg, 
                 accuracy_test_attack_3_avg, accuracy_test_clean_3_avg], f)
    

################################# Plotting Training Results #######################################
labels = ['ACCs before attack', 'ACCs before attack', 
          'ACCs after attack', 'ACCs after attack', 
          'ACCs after CryptoGraph', 'ACCs after CryptoGraph']
values = [accuracy_test_attack_1_avg, accuracy_test_clean_1_avg, 
          accuracy_test_attack_2_avg, accuracy_test_clean_2_avg, 
          accuracy_test_attack_3_avg, accuracy_test_clean_3_avg]
std_devs = [accuracy_test_attack_1_std, accuracy_test_clean_1_std, 
            accuracy_test_attack_2_std, accuracy_test_clean_2_std, 
            accuracy_test_attack_3_std, accuracy_test_clean_3_std]

# Define the x-coordinates of the bars and the width of the bars
x = np.arange(len(labels)//2)
width = 0.35

# Create the bar chart with error bars
bars1 = plt.bar(x - width/2, values[::2], width, yerr=std_devs[::2], label='Attack set', color='red', capsize=5)
bars2 = plt.bar(x + width/2, values[1::2], width, yerr=std_devs[1::2], label='Clean set', color='blue', capsize=5)

# Add a title and labels to the axes
plt.title('Average Accuracies')
plt.xlabel('Levels of Attack and Defense')
plt.ylabel('Average Accuracy')

# Make the x-labels vertical
plt.xticks(x, labels[::2])
# Add a legend
plt.legend()
# Add exact values on each bar
for bars in [bars1, bars2]:
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Adjust the layout to prevent the labels from being cut off
plt.tight_layout()
# Display the chart
plt.show()                